In [11]:
import pymc as pm
import numpy as np
import arviz as az
from aesara.tensor import subtensor as st

%load_ext lab_black
%load_ext watermark

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


# Multinomial Regression

Adapted from [unit 7: NHANESmulti.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/NHANESmulti.odc)

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/paraguay.csv).

## Associated lecture video: Unit 7 Lesson 17

In [3]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=79" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement

The National Health and Nutrition Examination Survey (NHANES) is a program of studies designed to assess the health and nutritional status of adults and children in the United States. The survey is unique in that it combines interviews and physical examinations. 

Assume that N subjects select a choice form K categories. The i-th subject is characterized by 3 covariates x[i,1], x[i,2], and x[i,3]. Given the covariates, model the probability of a subject selecting the category k, k=1,...,K.


In [8]:
# data
# fmt: off
y = np.array([[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1],
              [0, 0, 0, 1, 0]])

X =np.array([[2, 4, 9],
             [1, 5, 10],
             [1, 6, 14],
             [2, 4, 21],
             [2, 4, 22],
             [2, 6, 30],
             [3, 3, 33],
             [3, 2, 36],
             [3, 1, 40],
             [4, 1, 44]])
# fmt: on

In [9]:
y.shape

(10, 5)

In [33]:
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
X_aug.shape

(10, 4)

In [39]:
x_t = np.ones((10, 4))
b_t = np.arange(20).reshape((4, 5))
print(x_t.shape, b_t.shape)

np.dot(x_t, b_t)

(10, 4) (4, 5)


array([[30., 34., 38., 42., 46.],
       [30., 34., 38., 42., 46.],
       [30., 34., 38., 42., 46.],
       [30., 34., 38., 42., 46.],
       [30., 34., 38., 42., 46.],
       [30., 34., 38., 42., 46.],
       [30., 34., 38., 42., 46.],
       [30., 34., 38., 42., 46.],
       [30., 34., 38., 42., 46.],
       [30., 34., 38., 42., 46.]])

In [47]:
with pm.Model() as m:
    y_data = pm.Data("y", y, mutable=False)
    X_data = pm.Data("X", X_aug, mutable=True)

    _beta = pm.Normal("_beta", 0, tau=0.1, shape=(4, 5))
    # first col in BUGS example is assigned 0
    # might be better to prepend a column of zeroes rather than overwrite
    beta = pm.Deterministic("beta", st.set_subtensor(_beta[:, 0], 0))
    eta = pm.math.exp(pm.math.dot(X_data, beta))
    p = eta / pm.math.sum(eta, axis=1)

    pm.Multinomial("likelihood", n=1, p=p, observed=y)

    trace = pm.sample(10000)

Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [_beta]
INFO:pymc:NUTS: [_beta]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 37 seconds.
INFO:pymc:Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 37 seconds.


In [48]:
X_new = np.array([1, 3, 3, 30]).reshape((1, 4))

with m:
    pm.set_data({"X": X_new})
    ppc = pm.sample_posterior_predictive(trace)

az.summary(ppc)

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/stats.py:1366: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"likelihood[0, 0]",0.009,0.093,0.0,0.0,0.000,0.000,37449.0,37449.0,1.0
"likelihood[0, 1]",0.039,0.193,0.0,0.0,0.001,0.001,38328.0,38328.0,1.0
"likelihood[0, 2]",0.117,0.322,0.0,1.0,0.002,0.001,39586.0,39586.0,1.0
"likelihood[0, 3]",0.680,0.467,0.0,1.0,0.002,0.002,37059.0,37059.0,1.0
"likelihood[0, 4]",0.155,0.362,0.0,1.0,0.002,0.001,38297.0,38297.0,1.0


In [51]:
az.summary(ppc)["mean"].sum()

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/stats.py:1366: UserWarning: Selecting first found group: posterior_predictive
  warnings.warn(f"Selecting first found group: {data.groups()[0]}")


1.0

In [49]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

arviz : 0.12.1
numpy : 1.22.4
pymc  : 4.0.0
aesara: 2.6.6

